## Download Required Dependencies

In [ ]:
import subprocess
import sys

# Function to install a package using pip
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    "scikit-image",
    "numpy",
    "matplotlib",
    "opencv-python",
    "roboflow",
    
]

# Check and install each package
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        install_package(package)

## Implementation of HOG Feature Descriptor

Implementation of HOG Feature Descriptor involves the following steps:
1. Preprocess the data
2. Calculate and Visualize the Histogram of Gradients


### Step 1: Preprocess the data

In [ ]:
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2

image_BGR = cv2.imread("./media/puppy.jpg")
image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)

gray_image = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2GRAY) 

resized_image = resize(image=image_RGB, output_shape=(128,64))
resized_gray_image = resize(image=gray_image, output_shape=(128,64))

fig, axes = plt.subplots(2, 2)
axes[0, 0].imshow(image_RGB)
axes[0, 1].imshow(resized_image)
axes[1, 0].imshow(gray_image, cmap='gray')
axes[1, 1].imshow(resized_gray_image, cmap='gray')


### Step 2: Calculate and Visualize the Histogram of Gradients


In [ ]:
from skimage.feature import hog
from skimage import exposure

import cv2

fd, hog_image = hog(resized_gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 4), sharex=True, sharey=True)

ax1.imshow(resized_image)
ax1.set_title('Input image')

ax2.imshow(resized_gray_image, cmap=plt.cm.gray)
ax2.set_title('Grayscale image') 

ax3.imshow(hog_image, cmap=plt.cm.gray)
ax3.set_title('Histogram of Oriented Gradients') 





## Implementation of eXtended Center-Symmetric Local Binary Pattern (XCS-LBP) Feature Descriptor

### Step 1: Calculate XCS-LBP operator applied to the whole grayscale image

In [19]:
image_BGR = cv2.imread("./media/puppy.jpg")
image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
gray_image = cv2.cvtColor(image_RGB, cv2.COLOR_BGR2GRAY) 

def valid_index(image, x, y):
    return (x >= 0 and y >= 0 and x < image.shape[0] and y < image.shape[1])

def threshold_function(x):
    return 1 if x >= 0 else 0

def calculate_xcs_lbp_value(x, y, image, neighborhood_size=8):
    result = 0
    mask = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]
    half_neighborhood_size = int(neighborhood_size/2)
    image=image.astype('float32')
    for i in range(half_neighborhood_size):
        index1 = (x+mask[i][0], y+mask[i][1])
        index2 = (x+mask[i+half_neighborhood_size][0], y+mask[i+half_neighborhood_size][1])
        value1 = image[index1[0], index1[1]] if valid_index(image, index1[0], index1[1]) else 0
        value2 = image[index2[0], index2[1]] if valid_index(image, index2[0], index2[1]) else 0

        g1 = (value1-value2) + image[x, y]
        g2 = (value1-image[x, y])*(value2-image[x, y])
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]
        result += (threshold_function(g1+g2)*power_val[i])
    return result

xcs_lbp_image = gray_image.copy()
for i in range(xcs_lbp_image.shape[0]):
    for j in range(xcs_lbp_image.shape[1]):
        xcs_lbp_image[i, j] = calculate_xcs_lbp_value(i, j, gray_image)


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,8))
ax1.imshow(image_RGB)
ax1.set_title("Input Image")
ax2.imshow(gray_image, cmap='gray')
ax2.set_title("Grayscale Image")
ax3.imshow(xcs_lbp_image, cmap='gray')
ax3.set_title("XCS-LBP")
